In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA

# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['items.csv', 'sample_submission.csv', 'test.csv', 'sales_train.csv', 'item_categories.csv', 'shops.csv']


In [2]:
DATA_FNAME='text_features.h5'

In [3]:
items = pd.read_csv('../input/items.csv')
item_categories = pd.read_csv('../input/item_categories.csv')
shops = pd.read_csv('../input/shops.csv')

In [4]:
item_categories.head()

,item_category_name,item_category_id
0,PC - Гарнитуры/Наушники,0
1,Аксессуары - PS2,1
2,Аксессуары - PS3,2
3,Аксессуары - PS4,3
4,Аксессуары - PSP,4


In [5]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn.preprocessing import normalize


def _get_text_features(names, num_features):
    tfidf = TfidfVectorizer()
    features = tfidf.fit_transform(names).toarray()
    print('Num features from tfidf', features.shape[1])
    if features.shape[1] <= num_features or features.shape[0] <= num_features:
        return features.astype(np.float32)

    print('Using PCA')
    pca = PCA(n_components=num_features)
    return pca.fit_transform(normalize(features)).astype(np.float32)


def _get_text_feature_df(df, id_col, text_col, num_features):
    features = _get_text_features(df[text_col].tolist(), num_features)
    cols = ['{}_text_{}'.format(text_col, i) for i in range(features.shape[1])]
    output_df = pd.DataFrame(features, columns=cols).astype(np.float32)
    output_df[id_col] = df[id_col].tolist()
    return output_df


def get_item_text_features(items_df, num_features):
    return _get_text_feature_df(items_df, 'item_id', 'item_name', num_features)


def get_shop_text_features(shops_df, num_features):
    return _get_text_feature_df(shops_df, 'shop_id', 'shop_name', num_features)


def get_category_text_features(categories_df, num_features):
    return _get_text_feature_df(categories_df, 'item_category_id', 'item_category_name', num_features)


In [6]:
get_category_text_features(item_categories, 100).to_hdf(DATA_FNAME,'category_tfidf')
get_category_text_features(item_categories, 60).to_hdf(DATA_FNAME,'category_60')


Num features from tfidf 98
Num features from tfidf 98
Using PCA


In [7]:
get_shop_text_features(shops, 100).to_hdf(DATA_FNAME,'shop_tfidf')
get_shop_text_features(shops, 50).to_hdf(DATA_FNAME,'shop_50')


Num features from tfidf 113
Num features from tfidf 113
Using PCA


In [8]:
get_item_text_features(items, 500).to_hdf(DATA_FNAME,'item_500')

Num features from tfidf 18222
Using PCA
